In [44]:
import pandas as pd
import numpy as np

In [45]:
songToTrackDf = pd.read_csv("./taste_profile_song_to_tracks.txt", names=["song", "track"], sep="\t")
songToTrackDf.dropna(inplace=True)

In [46]:
songToTrackDf.head(5)

,song,track
0,SOAAADD12AB018A9DD,TRNCENP12903C9EF3A
1,SOAAADE12A6D4F80CC,TRSKKFK128F148B615
2,SOAAADF12A8C13DF62,TRCQMSP128F428A6F7
3,SOAAADZ12A8C1334FB,TRMDNZY128F425A532
4,SOAAAFI12A6D4F9C66,TRZEXLQ128F1491D17


In [47]:
userSongCountDf = pd.read_csv("./train_triplets.txt", names=["user", "song", "count"], sep="\t")

In [48]:
userSongCountDf.head(5)

,user,song,count
0,b80344d063b5ccb3212f76538f3d9e43d87dca9e,SOAKIMP12A8C130995,1
1,b80344d063b5ccb3212f76538f3d9e43d87dca9e,SOAPDEY12A81C210A9,1
2,b80344d063b5ccb3212f76538f3d9e43d87dca9e,SOBBMDR12A8C13253B,2
3,b80344d063b5ccb3212f76538f3d9e43d87dca9e,SOBFNSP12AF72A0E22,1
4,b80344d063b5ccb3212f76538f3d9e43d87dca9e,SOBFOVM12A58A7D494,1


In [49]:
mismatches = []
with open("sid_mismatches.txt", "r", encoding='UTF-8') as f:
    for line in f:
        mismatches.append((line[8:8+len(songToTrackDf["song"][0])], line[9+len(songToTrackDf["song"][0]):9+len(songToTrackDf["song"][0])+len(songToTrackDf["track"][0])]))
assert len(mismatches[0][0]) == len(songToTrackDf["song"][0])
assert len(mismatches[0][1]) == len(songToTrackDf["track"][0])
mismatchesDf = pd.DataFrame(mismatches, columns=["song", "track"])

In [50]:
mismatchesDf.head(5)

,song,track
0,SOUMNSI12AB0182807,TRMMGKQ128F9325E10
1,SOCMRBE12AB018C546,TRMMREB12903CEB1B1
2,SOLPHZY12AC468ABA8,TRMMBOC12903CEB46E
3,SONGHTM12A8C1374EF,TRMMITP128F425D8D0
4,SONGXCA12A8C13E82E,TRMMAYZ128F429ECE6


In [51]:
print(len(songToTrackDf))
songToTrackDf = songToTrackDf[(~songToTrackDf["track"].isin(mismatchesDf["track"])) & (~songToTrackDf["song"].isin(mismatchesDf["song"]))]
print(len(songToTrackDf))

383833
379962


In [52]:
print(len(userSongCountDf))
userSongCountDf = userSongCountDf[userSongCountDf["song"].isin(songToTrackDf["song"])]
print(len(userSongCountDf))

48373586
45795100


In [53]:
userSongTrackCountDf = userSongCountDf.merge(songToTrackDf, how="left", on="song")
print(len(userSongTrackCountDf))

45795100


In [54]:
userSongTrackCountDf.head(5)

,user,song,count,track
0,b80344d063b5ccb3212f76538f3d9e43d87dca9e,SOAKIMP12A8C130995,1,TRIQAUQ128F42435AD
1,b80344d063b5ccb3212f76538f3d9e43d87dca9e,SOAPDEY12A81C210A9,1,TRIRLYL128F42539D1
2,b80344d063b5ccb3212f76538f3d9e43d87dca9e,SOBBMDR12A8C13253B,2,TRMHBXZ128F4238406
3,b80344d063b5ccb3212f76538f3d9e43d87dca9e,SOBFNSP12AF72A0E22,1,TRYQMNI128F147C1C7
4,b80344d063b5ccb3212f76538f3d9e43d87dca9e,SOBFOVM12A58A7D494,1,TRAHZNE128F9341B86


In [55]:
userSongTrackCountDf = userSongTrackCountDf[userSongTrackCountDf["count"] > 1]
uniqueSongs = userSongTrackCountDf["track"].unique()
print(len(userSongTrackCountDf), len(userSongTrackCountDf["track"].unique()))

18550228 327982


In [57]:
print(uniqueSongs[:5])

['TRMHBXZ128F4238406' 'TRPLAXZ128F4292406' 'TRPTWGR128F1452734'
 'TRHNGRY128E0791670' 'TRYSBOR128F92CA258']


In [58]:
# this is the track of all unique songs 
pd.Series(uniqueSongs).to_csv("allSongs.csv", header=None, index=False)

In [ ]:
userSongTrackCountDf.to_csv("userSongTrackCount.csv", header=["user", "song", "count", "track"], index=False)

In [78]:
tmp = userSongTrackCountDf[["user", "track"]]
userSongListDf = tmp.groupby("user")["track"].apply(list).reset_index(name='tracks')

In [82]:
userSongListDf.head(5)

,user,tracks
0,00000b722001882066dff9d2da8a775658053ea0,[TRTIKBI128F146A471]
1,00001638d6189236866af9bbf309ae6c2347ffdc,"[TRZVSYU128EF34065D, TRSUNFH128E078312D, TRWEA..."
2,0000175652312d12576d9e6b84f600caa24c4715,"[TRLACNL128F148D241, TRONLGT128F424BEE0, TRRAX..."
3,00001cf0dce3fb22b0df0f3a1d9cd21e38385372,"[TRZPDFG128F92FDEF5, TRKABMK128F426BF72, TRLVV..."
4,0000267bde1b3a70ea75cf2b2d216cb828e3202b,"[TRZSLAK128F149F9B0, TRZXSSH128F4284839]"


In [81]:
userSongListDf.to_json("userSongList.json", orient="records")